In [ ]:
import tensorflow as tf
import tensorflow_io as tfio
import os 
import tensorflow as tf
import matplotlib.pyplot as plt
import os 
import numpy as np
from tensorflow import keras
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from methods_audio import data_handling
from methods_audio import data_augmentation
from methods_audio import denoising 
from methods_audio import model_performance_training
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, TimeMask, SpecCompose, SpecFrequencyMask, Mp3Compression
import random
from methods_audio import data_handling

In [2]:
data = data_handling.get_data()

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [ ]:
len(data)

# 2. Read data (transforming file names into waves) <br>
Additionally, the mean is removed and the data is normalized. 

In [3]:
data = data.map(data_handling.read_in_data) 

# 3. Get input for model training 

In [4]:
samples, labels = data_handling.extract_samples_labels(data)

2023-05-17 14:31:38.993986: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Get data

In [11]:
from sklearn.model_selection import StratifiedKFold

splits = 2

kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=123)   

for train, test in kfold.split(samples, labels):
    # 3. Split data and transform it from list of numpy.ndarray into numpy.ndarray of numpy.ndarray
    x_train = np.array(samples)[train.astype(int)]
    y_train = np.array(labels)[train.astype(int)]
    x_valid = np.array(samples)[test.astype(int)]
    y_valid = np.array(labels)[test.astype(int)]

    # 4. Transform numpy.ndarray into lists
    x_train_list = x_train.tolist() 
    y_train_list = y_train.tolist()
    x_valid_list = x_valid.tolist()

/var/folders/bz/rfsj8qqx5zsg0g1p0xdb1fkw0000gn/T/ipykernel_57413/484824578.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train = np.array(samples)[train.astype(int)]
/var/folders/bz/rfsj8qqx5zsg0g1p0xdb1fkw0000gn/T/ipykernel_57413/484824578.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_valid = np.array(samples)[test.astype(int)]


In [12]:
x_train_list[:1]


[array([-0.19126989, -0.22592975, -0.12772681, ...,  0.05520026,
         0.16174725,  0.19448157], dtype=float32)]

In [18]:
def time_gaussian_noise(samples, probability): 
    # Similar to Noise Adds
    sample_rate = 8000
    augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=probability),
])  
    augmented_samples = augment(samples=samples, sample_rate=sample_rate)
    return augmented_samples

def time_mask(samples, probability): 
    sample_rate = 8000
    augment = Compose([
    TimeMask(min_band_part=0.0, max_band_part= 0.1, fade = False, p = probability), 
])  
    augmented_samples = augment(samples=samples, sample_rate=sample_rate)
    return augmented_samples

def time_compression(samples, probability): 
    # Similar to gain
    sample_rate = 8000
    augment = Compose([  
    Mp3Compression(p = probability),
]) 
    augmented_samples = augment(samples=samples, sample_rate=sample_rate)
    return augmented_samples

def time_augmentation(samples, labels, probability): 
    
    new_samples = []
    new_labels = []
    for sample, label in zip(samples, labels): 
        new_samples += [sample]
        new_labels += [label]

        print('meli')
        compressed_sample = time_compression(sample, probability)
        print(compressed_sample)

        if not np.all([np.array_equal(arr1, arr2) for arr1, arr2 in zip(sample, compressed_sample)]): 
            # if the samples are not the same, it means that augmentation occured so save the new one 
            new_samples += [compressed_sample]
            new_labels += [label]


    # Shuffle the lists to reduce any type of bias, ensuring that the paring of signal/label is kept 
    paired_list = list(zip(new_samples, new_labels))
    random.shuffle(paired_list)

    shuffled_signals, suffled_labels = zip(*paired_list) #unziping 

    # Eventhough we unzipped the data, the type is still duple, so when returning it, we cast it to list

    return list(shuffled_signals), list(suffled_labels)

In [19]:
probability = 1
x_train_list2, y_train_list2 = time_augmentation(x_train_list[:1], y_train_list[:1], probability)

meli


/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


FileNotFoundError: [Errno 2] No such file or directory: 'ffmpeg'

In [ ]:
len(x_train_list)

In [ ]:
list1 = x_train_list[0]
list2 = x_train_list[1]
np.all([np.array_equal(arr1, arr2) for arr1, arr2 in zip(list1, list2)])


In [ ]:
def given_list_return_30(list):
    return list[30]

In [ ]:
def take_lists (acc, pre, rec, loss): 
    print(given_list_return_30(acc))
    print(given_list_return_30(pre))
    print(given_list_return_30(rec))
    print(given_list_return_30(loss))

In [ ]:
accuracy = [0.5053533017635345,
0.6556296408176422,
0.7123773217201232,
0.7079538822174072,
0.8207761287689209,
0.8328765153884887,
0.8795721054077148,
0.8891479969024658,
0.9034916043281556,
0.9020764231681824,
0.9299391746520996,
0.9183966159820557,
0.9034844636917114,
0.9380990862846375,
0.9383819580078125,
0.9271206855773926,
0.9456948041915894,
0.9428842425346374,
0.9414769887924195,
0.9451345920562744,
0.9538555026054383,
0.9580765008926392,
0.9485113024711609,
0.9561082482337951,
0.9426045417785645,
0.9549822568893432,
0.9569513082504273,
0.9527315139770508,
0.9563903212547302,
0.9372456908226013,
0.9358582377433777,
0.9558281302452087,
0.9648303270339966,
0.9547021627426148,
0.9625791788101197,
0.9597642660140991,
0.9614544034004211,
0.9631413698196412,
0.9642681241035461,
0.9625787734985352,
0.9608902335166931,
0.9606101036071777,
0.9645494222640991,
0.9667993783950806,
0.9639864444732666,
0.9645494222640991,
0.9620173811912537,
0.9651116132736206,
0.9656734108924866,
0.9634238600730896]

In [ ]:
precision = [0.598873633146286,
0.7381534159183503,
0.7508548498153687,
0.6987374782562256,
0.8254450678825378,
0.8001165390014648,
0.883429765701294,
0.862846040725708,
0.9176298260688782,
0.9052736639976502,
0.9341355919837951,
0.9443249464035034,
0.8896600246429444,
0.9275087118148804,
0.9344295978546142,
0.9220501899719238,
0.9639141440391541,
0.9249795079231262,
0.9444829821586609,
0.9354500055313111,
0.9481643199920654,
0.9540666103363037,
0.9526579022407532,
0.9497284293174744,
0.9253448128700257,
0.9456703662872314,
0.9454304814338684,
0.9691927671432495,
0.9482736110687255,
0.9520818829536438,
0.9746097803115845,
0.9508116483688355,
0.9730987548828125,
0.9623815655708313,
0.9604140639305114,
0.9586260080337524,
0.9577802538871765,
0.9611108064651489,
0.9640872836112976,
0.9593278765678406,
0.9578320741653442,
0.9554691195487977,
0.9589033126831055,
0.9658541440963745,
0.9605337619781494,
0.9639965891838074,
0.9556813836097717,
0.9631246328353882,
0.9646394371986389,
0.9586552381515503]




In [ ]:
recall = [0.8107360668480397,
0.7525423645973206,
0.7723927736282349,
0.8752988100051879,
0.8610392332077026,
0.9175666570663452,
0.8768711805343627,
0.9350409865379333,
0.8926760792732239,
0.908487331867218,
0.9249017238616943,
0.8932346820831298,
0.9305164337158203,
0.9508824944496155,
0.9435283064842224,
0.9412445425987244,
0.9265711903572083,
0.9655478715896606,
0.9412477254867554,
0.9581984639167785,
0.9604710817337037,
0.9627246141433716,
0.9452088832855224,
0.9638529539108276,
0.9661223888397217,
0.9666730403900147,
0.9706310153007507,
0.9356075644493103,
0.9666730523109436,
0.9231702089309692,
0.8962504982948303,
0.9621564507484436,
0.9559481263160705,
0.9469260931015014,
0.9649828910827637,
0.9610280990600586,
0.9655478715896606,
0.9655415296554566,
0.9644163370132446,
0.9666714668273926,
0.9644163370132446,
0.9672364234924317,
0.9706294417381287,
0.9678030014038086,
0.9678077578544617,
0.9649876713752746,
0.9700596928596497,
0.9672427892684936,
0.9666746377944946,
0.9689313530921936]

In [ ]:
loss = [0.7029196858406067,
0.598166024684906,
0.5438540041446686,
0.5108146131038666,
0.4023751258850098,
0.3613684594631195,
0.28302874565124514,
0.26664240956306456,
0.2444238245487213,
0.2399083971977234,
0.18699799180030824,
0.21292608380317687,
0.23310189247131347,
0.16458430588245393,
0.16046417951583863,
0.18385301232337953,
0.15592080950737,
0.15015002191066742,
0.15618603378534318,
0.14431273341178893,
0.12665585279464722,
0.12672632932662964,
0.1345878556370735,
0.13021054863929749,
0.15255046039819717,
0.1329212889075279,
0.12572951465845109,
0.13774742037057877,
0.13041517734527588,
0.1793070673942566,
0.19231543987989425,
0.12929256856441498,
0.11244143098592758,
0.12843812704086305,
0.1160872682929039,
0.12212135046720504,
0.11703045964241028,
0.1188506841659546,
0.11395584344863892,
0.12957920879125595,
0.12389957308769226,
0.12683125138282775,
0.11255563497543335,
0.11432499587535858,
0.12230847179889678,
0.12158175110816956,
0.1277119681239128,
0.11979437470436097,
0.11951465606689453,
0.12347009629011155] 

In [ ]:
take_lists(accuracy, precision, recall, loss)